the Claim of the competition is we have to 'shorten the test time'  Y-pred !
----

forked john, since he has made wonderful graphs
now the mirakle is when you make a kind of nearly perfect prediction, you get a score LB 0.48
----

Remark the current solutions don't minimize the time for mercedes..
To minimize the time we have to do other math then regressions

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb 
from sklearn.metrics import r2_score

%matplotlib inline

from IPython.display import display, HTML
# Shows all columns of a dataframe
def show_dataframe(X, rows = 2):
    display(HTML(X.to_html(max_rows=rows)))

In [ ]:
# Datasets
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

The combined X0 - X5 combination labellizing improves forecast
----
evidently, the more parameters the better the regression fit, thats no miracle


In [ ]:
#add X0-8 combinations
for xi in ['X1','X2','X3','X4','X5','X6','X8']:
    nieuwveld='X0'+xi
    train[nieuwveld]=train['X0']+'-'+train[xi]
    test[nieuwveld]=test['X0']+'-'+test[xi]

# Categorical features
cat_cols = []
for c in train.columns:
    if train[c].dtype == 'object':
        cat_cols.append(c)
print('Categorical columns:', cat_cols)

# Dublicate features
d = {}; done = []
cols = train.columns.values
for c in cols: d[c]=[]
for i in range(len(cols)):
    if i not in done:
        for j in range(i+1, len(cols)):
            if all(train[cols[i]] == train[cols[j]]):
                done.append(j)
                d[cols[i]].append(cols[j])
dub_cols = []
for k in d.keys():
    if len(d[k]) > 0: 
        # print k, d[k]
        dub_cols += d[k]        
print('Dublicates:', dub_cols)

# Constant columns
const_cols = []
for c in cols:
    if len(train[c].unique()) == 1:
        const_cols.append(c)
print('Constant cols:', const_cols)

Figures below show categorical features (on the left) sorted by means of **y**'s grouped by labels. On the right there are corresponding **mean**'s, **std**'s (filled blue), **max**'s (green line) and **min**'s (red line).

In [ ]:
# Glue train + test
train['eval_set'] = 0; test['eval_set'] = 1
df = pd.concat([train, test], axis=0, copy=True)
# Reset index
df.reset_index(drop=True, inplace=True)

### Categorical feature encoding
In the next cell for every categorical column from **cat_cols** we'll find **mean** of **y's** for every label using **.groupby()**. Then we sort labels by values of **means**. Now, when labels are sorted, they can be encoded by numbers from *0* to *numbers of labels - 1*.

I use here a quantile ranking
----

In [ ]:
def add_new_col(x):
    if x not in new_col.keys(): 
        # set n/2 x if is contained in test, but not in train 
        # (n is the number of unique labels in train)
        # or an alternative could be -100 (something out of range [0; n-1]
        return int(len(new_col.keys())/2)
    return new_col[x] # rank of the label

for c in cat_cols:
    # get labels and corresponding means
    new_col = train.groupby(c).y.quantile(q=0.5).sort_values().reset_index()
    # make a dictionary, where key is a label and value is the rank of that label
    new_col = new_col.reset_index().set_index(c).drop('y', axis=1)['index'].to_dict()
    # add new column to the dataframe
    df[c + '_new'] = df[c].apply(add_new_col)

# drop old categorical columns
df_new = df.drop(cat_cols, axis=1)

# show the result
show_dataframe(df_new, 5)

### Train-test split

In [ ]:
from scipy.stats import signaltonoise
X = df.drop(list((set(const_cols) | set(dub_cols) | set(cat_cols))), axis=1)

# Train
X_train = X[X.eval_set == 0]
y_train = X_train.pop('y'); 
X_train = X_train.drop(['eval_set', 'ID'], axis=1)

# Test
X_test = X[X.eval_set == 1]
X_test = X_test.drop(['y', 'eval_set', 'ID'], axis=1)

# Base score
#y_mean = y_train.sum() #q0.01 LB 0.5546 q=0.25 0.5549  #permut categ LB 0.51
y_mean = 1/signaltonoise(train['y'])
# Shapes

print('Shape X_train: {}\nShape X_test: {}'.format(X_train.shape, X_test.shape))

In [ ]:
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
kolom=X_train.columns
kolom=[w for w in kolom if w not in ['y']]

n_comp = 50


# PCA
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(X_train[kolom])
pca2_results_test = pca.transform(X_test[kolom])


#save columns list before adding the decomposition components

usable_columns = list(set(train.columns) - set(['y']))

# Append decomposition components to datasets
for i in range(1, n_comp + 1):
    X_train['pca_' + str(i)] = pca2_results_train[:, i - 1]
    X_test['pca_' + str(i)] = pca2_results_test[:, i - 1]



In [ ]:
### Regressor

# prepare dict of params for xgboost to run with
xgb_params = {
    'n_trees': 100, 
    'eta': 0.005,
    'max_depth': 3,
    'subsample': 0.95,
    'colsample_bytree': 0.6,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean,
    'silent': 1
}

# form DMatrices for Xgboost training
dtrain = xgb.DMatrix(X_train, np.log(y_train))
dtest = xgb.DMatrix(X_test)

# evaluation metric
def the_metric(y_pred, y):
    y_true = y.get_label()
    return 'r2', r2_score(y_true, y_pred)
# evaluation metric
def euclid(y_pred,y):
    y_true = y.get_label()
    xa=y_pred
    ya=y_true
    eu0=np.sqrt(sum(pow(a-b,2) for a, b in zip(xa, ya)))
    xb=y_pred
    yb=y_true
    eu1=np.sqrt(sum(pow(a-b,2) for a, b in zip(xb, yb)))
    return 'r2',eu0*eu1

# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=3000, # increase to have better results (~700)
                   nfold = 3,
                   early_stopping_rounds=50,
                   #feval=the_metric,
                   feval=euclid,
                   verbose_eval=100, 
                   show_stdv=False
                  )

num_boost_rounds = len(cv_result)
print('num_boost_rounds=' + str(num_boost_rounds))

# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

# Predict on trian and test
y_train_pred = np.exp(model.predict(dtrain))
y_pred = np.exp(model.predict(dtest))

print('First 5 predicted test values:', y_pred[:5])



In [ ]:
plt.figure(figsize=(16,4))

plt.subplot(1,4,1)
#train_scores = cv_result['train-rmse']
#train_stds = cv_result['train-r2-std']
#plt.plot(train_scores, color='green')
#plt.fill_between(range(len(cv_result)), train_scores - train_stds, train_scores + train_stds, alpha=0.1, color='green')
#test_scores = cv_result['train-rmse']
#test_stds = cv_result['test-r2-std']
#plt.plot(test_scores, color='red')
#plt.fill_between(range(len(cv_result)), test_scores - test_stds, test_scores + test_stds, alpha=0.1, color='red')
plt.title('Train and test cv scores (R2)')

plt.subplot(1,4,2)
plt.title('True vs. Pred. train')
plt.plot([80,265], [80,265], color='g', alpha=0.3)
plt.scatter(x=y_train, y=y_train_pred, marker='.', alpha=0.5)
plt.scatter(x=[np.mean(y_train)], y=[np.mean(y_train_pred)], marker='o', color='red')
plt.xlabel('Real train'); plt.ylabel('Pred. train')

plt.subplot(1,4,3)
sns.distplot(y_train, kde=False, color='g')
sns.distplot(y_train_pred, kde=False, color='r')
plt.title('Distr. of train and pred. train')

plt.subplot(1,4,4)
sns.distplot(y_train, kde=False, color='g')
sns.distplot(y_pred, kde=False, color='b')
plt.title('Distr. of train and pred. test')



plt.figure(figsize=(18,1))
plt.plot(y_train_pred[:200], color='r', linewidth=0.7)
plt.plot(y_train[:200], color='g', linewidth=0.7)
plt.title('First 200 true and pred. trains')

print('Mean error =', np.mean(y_train - y_train_pred))
print('Train r2 =', r2_score(y_train, y_train_pred))

In [ ]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': y_pred})
output.to_csv('subm.csv', index=False)
fig = plt.figure(figsize=(15,10))
sns.jointplot(y_train, y_train_pred, kind="reg")